# Cleaning

In [1]:
#dependencies

import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import csv 
import os
import numpy as np

In [2]:
# Store CSV into DataFrame
districts = "Resources/districts.csv"
naep = "Resources/naep.csv"
states = "Resources/states.csv"


naep_pd = pd.read_csv(naep)
districts_pd = pd.read_csv(districts)
states_pd = pd.read_csv(states)





In [3]:
#Naep_pd focus:
#naep_states.csv: Aggregated results from the National Assessment of Educational Progress (NAEP) for states.
#focusing on 4th & 8th grades 
# year ranges from 1990-2017, but missing 91, 93, 95, 97,99, 2001, 05, 06, 08, 10, 14, 16: total 15 years  

#print(naep_pd)
#naep_pd['YEAR'].unique()
#naep_pd['AVG_SCORE'].value_counts()
    # there is null values being shown as: "-- & ‡" on AVG_SCORE cloumn
    #To clean and change it to 0:
    #mask1 = (naep_pd['AVG_SCORE'] == '—')
    #naep_pd['AVG_SCORE'][mask1] = 0
    #mask2 = (naep_pd['AVG_SCORE'] == '‡')
    #naep_pd['AVG_SCORE'][mask2] = 0

#naep_pd['TEST_SUBJECT'].value_counts()
#naep_pd['TEST_YEAR'].value_counts() 
#naep_pd['TEST_YEAR'].unique()



#naep_pd.columns
naep_pd.head()



,YEAR,STATE,AVG_SCORE,TEST_SUBJECT,TEST_YEAR
0,2017,Alabama,232.170687741509,Mathematics,4
1,2017,Alaska,230.456277558902,Mathematics,4
2,2017,Arizona,234.435788152091,Mathematics,4
3,2017,Arkansas,233.848143678937,Mathematics,4
4,2017,California,232.262940994562,Mathematics,4


In [41]:
#districts_pd.head()
#Notes 
    # Revenue comes from income taxes per state, local taxes, fed funds, local misch
    # the data frame splits that: Total: fed, state, local
    # TCURINST: Total current spending for instruction
    # TCURSSVC: Total current spending for support services
    # TCURONON: Other current spending
    # TCAPOUT: Total capital outlay expenditure

# Ensuring values has no "unknown values"

#districts_pd['TOTALREV'].value_counts()
#districts_pd['TFEDREV'].value_counts()
#districts_pd['TLOCREV'].value_counts()
#districts_pd['TOTALEXP'].value_counts()
#districts_pd['TCURINST'].value_counts()
#districts_pd['TCURSSVC'].value_counts()
#districts_pd['TCURONON'].value_counts()
#districts_pd['TCAPOUT'].value_counts()



#rename 
districts_pd = districts_pd.rename(columns= {'YRDATA': 'YEAR'})
districts_pd.head()
#districts_pd.dtypes

#Checking the sum of the revenue fed, state & local & exp     
    
#districts_pd["TOTALREV"].sum()
    # 11605107285
    
#districts_pd["TSTREV"].sum()
    # 5385272201

#districts_pd["TLOCREV"].sum()
    # 5240915749
    
#districts_pd["TOTALEXP"].sum()
    #11737958705
    


,STATE,ENROLL,NAME,YEAR,TOTALREV,TFEDREV,TSTREV,TLOCREV,TOTALEXP,TCURINST,TCURSSVC,TCURONON,TCAPOUT
0,Alabama,9609.0,AUTAUGA COUNTY SCHOOL DISTRICT,2016,80867,7447,53842,19578,76672,43843,23941,6401.0,1506
1,Alabama,30931.0,BALDWIN COUNTY SCHOOL DISTRICT,2016,338236,23710,145180,169346,299880,164977,97231,19439.0,9749
2,Alabama,912.0,BARBOUR COUNTY SCHOOL DISTRICT,2016,10116,2342,5434,2340,10070,4907,3896,975.0,110
3,Alabama,2842.0,EUFAULA CITY SCHOOL DISTRICT,2016,26182,3558,15900,6724,29843,15302,7901,2274.0,4133
4,Alabama,3322.0,BIBB COUNTY SCHOOL DISTRICT,2016,32486,3664,21846,6976,31662,16407,11087,3122.0,486


In [50]:
states_pd.head()
    # State vs district csv files have different data --
    # district covers more because of because of different districts within a state, where state csv covers more macro

#cleaning 

#states_pd['STATE'].value_counts()
    # each state shows 25 times

#states_pd['YEAR'].value_counts()
    # each year shows up 51 times, 19992-2016

#states_pd['TOTAL_REVENUE'].value_counts()
#states_pd['FEDERAL_REVENUE'].value_counts()
#states_pd['STATE_REVENUE'].value_counts()
    # There's 0 25's
#states_pd['LOCAL_REVENUE'].value_counts()
#states_pd['TOTAL_EXPENDITURE'].value_counts()
#states_pd['INSTRUCTION_EXPENDITURE'].value_counts()
#states_pd['SUPPORT_SERVICES_EXPENDITURE'].value_counts()
#states_pd['OTHER_EXPENDITURE'].value_counts()
#states_pd['CAPITAL_OUTLAY_EXPENDITURE'].value_counts()

#Checking the total rev & expenses
    # -- will get us an understanding which values or columns to keep 
    
#states_pd["TOTAL_REVENUE"].sum()
    #11605107285

#states_pd["FEDERAL_REVENUE"].sum()
    #978919327
    
#states_pd["LOCAL_REVENUE"].sum()
    #5240915749

#states_pd["TOTAL_EXPENDITURE"].sum()
    #11737958705







,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
0,Alabama,1992,NaN,2678885,304177,1659028,715680,2653798,1481703,735036,NaN,174053
1,Alaska,1992,NaN,1049591,106780,720711,222100,972488,498362,350902,NaN,37451
2,Arizona,1992,NaN,3258079,297888,1369815,1590376,3401580,1435908,1007732,NaN,609114
3,Arkansas,1992,NaN,1711959,178571,958785,574603,1743022,964323,483488,NaN,145212
4,California,1992,NaN,26260025,2072470,16546514,7641041,27138832,14358922,8520926,NaN,2044688


In [20]:
# merging naep and districts 
#education_df = pd.merge(naep_pd, districts_pd, on = ["YEAR", "STATE"])

education_df.head()

#education_df['ENROLL'].value_counts()
    # O: no recrods of the students enrolled in that district -- 38369

#education_df['YEAR'].value_counts()
    # 94' - 2015
    
#education_df['STATE'].value_counts()
    #dc is there
    
#education_df['TEST_SUBJECT'].value_counts()
    # consistent: Math & reading 

#education_df['TEST_YEAR'].value_counts()
    #consistent: 4th & 8th grade
    





,YEAR,STATE,AVG_SCORE,TEST_SUBJECT,TEST_YEAR,ENROLL,NAME,TOTALREV,TFEDREV,TSTREV,TLOCREV,TOTALEXP,TCURINST,TCURSSVC,TCURONON,TCAPOUT
0,2015,Alabama,230.977422662321,Mathematics,4,9664.0,AUTAUGA COUNTY SCHOOL DISTRICT,79665,7574,53244,18847,76069,44085,23217,6580.0,691
1,2015,Alabama,230.977422662321,Mathematics,4,30596.0,BALDWIN COUNTY SCHOOL DISTRICT,330317,23602,143282,163433,297083,155668,99682,19535.0,13256
2,2015,Alabama,230.977422662321,Mathematics,4,925.0,BARBOUR COUNTY SCHOOL DISTRICT,10519,2518,5632,2369,10314,5249,3835,1018.0,0
3,2015,Alabama,230.977422662321,Mathematics,4,2829.0,EUFAULA CITY SCHOOL DISTRICT,26076,3374,16048,6654,25602,14887,7494,2184.0,990
4,2015,Alabama,230.977422662321,Mathematics,4,3357.0,BIBB COUNTY SCHOOL DISTRICT,31825,3586,21687,6552,31293,16019,10822,2950.0,858


In [53]:
# merge education and states 
#education_finances_df = pd.merge(education_df, states_pd, on = ["STATE", "YEAR", "ENROLL"])

#education_finances_df.head()

education_finances_df.columns

# some columns repeat each other with different values
    #how do we determine which stays and which drops/gets hidden
    
    # -- checking both state & disrticts -- they have consistent total revenue & expenditures
     # but the only issue that the the state & local revenue is different for both csv files. 
        # the state covers state more & district has different districts. 
        # We can keep the state data on state & local revenue and expenses as well
    

Index(['YEAR', 'STATE', 'AVG_SCORE', 'TEST_SUBJECT', 'TEST_YEAR', 'ENROLL',
       'NAME', 'TOTALREV', 'TFEDREV', 'TSTREV', 'TLOCREV', 'TOTALEXP',
       'TCURINST', 'TCURSSVC', 'TCURONON', 'TCAPOUT', 'TOTAL_REVENUE',
       'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE',
       'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
       'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE',
       'CAPITAL_OUTLAY_EXPENDITURE'],
      dtype='object')

In [57]:
# cleaning up 
#education_finances_df = education_finances_df[['YEAR', 'STATE', 'NAME', 'AVG_SCORE', 'TEST_SUBJECT', 'TEST_YEAR', 'TOTAL_REVENUE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE']].copy()
#education_finances_df.head()

#education_finances_df = education_finances_df[[ "YEAR", "STATE", "NAME", "ENROLL", "AVG_SCORE", "TEST_SUBJECT", "TEST_YEAR", "TOTAL_REVENUE","INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE"]].copy()

education_finances_df.head(500)

,YEAR,STATE,NAME,ENROLL,AVG_SCORE,TEST_SUBJECT,TEST_YEAR,TOTAL_REVENUE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE
0,2015,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,46155.0,231.307525116183,Mathematics,4,1382282,557345,379854,41841.0
1,2015,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,46155.0,263.059776256545,Mathematics,8,1382282,557345,379854,41841.0
2,2015,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,46155.0,212.305347670353,Reading,4,1382282,557345,379854,41841.0
3,2015,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,46155.0,247.653444946724,Reading,8,1382282,557345,379854,41841.0
4,2015,Hawaii,HAWAII PUBLIC SCHOOLS,182384.0,237.895806837329,Mathematics,4,2703683,1377713,835930,145560.0
5,2015,Hawaii,HAWAII PUBLIC SCHOOLS,182384.0,279.340145034503,Mathematics,8,2703683,1377713,835930,145560.0
6,2015,Hawaii,HAWAII PUBLIC SCHOOLS,182384.0,215.124313046421,Reading,4,2703683,1377713,835930,145560.0
7,2015,Hawaii,HAWAII PUBLIC SCHOOLS,182384.0,257.35043981355,Reading,8,2703683,1377713,835930,145560.0
8,2013,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,44179.0,228.567253872576,Mathematics,4,1300036,559940,332997,39825.0
9,2013,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,44179.0,265.259365712508,Mathematics,8,1300036,559940,332997,39825.0


In [9]:
#education_finances_df.columns

education_finances_df = education_finances_df.rename(columns={'YEAR': 'Year',
                                                             'STATE': 'State',
                                                             'NAME': 'District Name',
                                                             'AVG_SCORE': 'Average Score',
                                                             'TEST_SUBJECT': 'Test Subject',
                                                             'TEST_YEAR': 'Grade Year',
                                                             'TOTAL_REVENUE': 'Total Revenue',
                                                             'INSTRUCTION_EXPENDITURE': 'Instructors Expenditures',
                                                             'SUPPORT_SERVICES_EXPENDITURE': 'Support Services Expenditures',
                                                             'OTHER_EXPENDITURE': 'Other Expenditures'})
education_finances_df.head()


,Year,State,District Name,Average Score,Test Subject,Grade Year,Total Revenue,Instructors Expenditures,Support Services Expenditures,Other Expenditures
0,2015,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,231.307525116183,Mathematics,4,1382282,557345,379854,41841.0
1,2015,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,263.059776256545,Mathematics,8,1382282,557345,379854,41841.0
2,2015,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,212.305347670353,Reading,4,1382282,557345,379854,41841.0
3,2015,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,247.653444946724,Reading,8,1382282,557345,379854,41841.0
4,2015,Hawaii,HAWAII PUBLIC SCHOOLS,237.895806837329,Mathematics,4,2703683,1377713,835930,145560.0


In [22]:
education_finances_df["State"].value_counts()

edu


#uk['region'].unique()

California              3258
Texas                   3153
Illinois                2841
Nebraska                2397
New York                2154
Ohio                    1986
Michigan                1848
Pennsylvania            1824
New Jersey              1809
Oklahoma                1809
Montana                 1668
Missouri                1626
Minnesota               1566
Iowa                    1323
Massachusetts           1317
Wisconsin               1281
Arkansas                1011
Vermont                  987
Indiana                  939
Kansas                   912
Oregon                   909
North Dakota             897
Washington               888
Maine                    870
Arizona                  681
Georgia                  597
Colorado                 582
South Dakota             537
Kentucky                 528
New Hampshire            528
Connecticut              498
Mississippi              486
Virginia                 477
Tennessee                417
North Carolina

In [11]:
education_finances_df.describe()

,Year,Grade Year,Total Revenue,Instructors Expenditures,Support Services Expenditures,Other Expenditures
count,47678.000000,47678.000000,4.767800e+04,4.767800e+04,4.767800e+04,74.000000
mean,1992.022400,5.334284,7.795256e+06,4.203842e+06,2.358849e+06,85800.175676
std,0.613214,1.885974,7.374191e+06,4.100572e+06,2.290069e+06,47475.496059
min,1992.000000,4.000000,5.516680e+05,2.834840e+05,1.523400e+05,23107.000000
25%,1992.000000,4.000000,2.396705e+06,1.283577e+06,7.350360e+05,39825.000000
50%,1992.000000,4.000000,4.993909e+06,2.598925e+06,1.478639e+06,78376.500000
75%,1992.000000,8.000000,1.081469e+07,6.042936e+06,3.321276e+06,133810.000000
max,2015.000000,8.000000,2.626002e+07,1.435892e+07,8.520926e+06,151243.000000


In [51]:
mask = (education_finances_df['Average Score'] == '—')
education_finances_df['Average Score'][mask] = 0

Series([], Name: Average Score, dtype: object)

In [52]:
education_finances_df.head(500)

,Year,State,District Name,Average Score,Test Subject,Grade Year,Total Revenue,Instructors Expenditures,Support Services Expenditures,Other Expenditures
0,2015,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,231.307525116183,Mathematics,4,1382282,557345,379854,41841.0
1,2015,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,263.059776256545,Mathematics,8,1382282,557345,379854,41841.0
2,2015,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,212.305347670353,Reading,4,1382282,557345,379854,41841.0
3,2015,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,247.653444946724,Reading,8,1382282,557345,379854,41841.0
4,2015,Hawaii,HAWAII PUBLIC SCHOOLS,237.895806837329,Mathematics,4,2703683,1377713,835930,145560.0
5,2015,Hawaii,HAWAII PUBLIC SCHOOLS,279.340145034503,Mathematics,8,2703683,1377713,835930,145560.0
6,2015,Hawaii,HAWAII PUBLIC SCHOOLS,215.124313046421,Reading,4,2703683,1377713,835930,145560.0
7,2015,Hawaii,HAWAII PUBLIC SCHOOLS,257.35043981355,Reading,8,2703683,1377713,835930,145560.0
8,2013,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,228.567253872576,Mathematics,4,1300036,559940,332997,39825.0
9,2013,District of Columbia,DISTRICT OF COLUMBIA PUBLIC SCHOOLS,265.259365712508,Mathematics,8,1300036,559940,332997,39825.0


In [53]:
#download as a csv file: 
education_finances_df.to_csv('education.csv', index=False)

In [ ]:
DROP TABLE education

CREATE TABLE education (
	"Year" INT NOT NULL,
	"State" VARCHAR(50) NOT NULL,
	"District Name" VARCHAR(50) NOT NULL, 
	"Average Score" FLOAT NOT NULL,
	"Test Subject" VARCHAR(225) NOT NULL,
	"Grade Year" INT NOT NULL,
	"Total Revenue" FLOAT NOT NULL,
	"Instructors Expenditures" FLOAT NOT NULL,
	"Support Services Expenditures" FLOAT NOT NULL,
	"Other Expenditures" FLOAT
);


SELECT * FROM education

In [68]:
education_finances_df.dtypes

Year                               int64
State                             object
District Name                     object
Average Score                     object
Test Subject                      object
Grade Year                         int64
Total Revenue                      int64
Instructors Expenditures           int64
Support Services Expenditures      int64
Other Expenditures               float64
dtype: object

In [99]:
naep_pd[['AVG_SCORE', 'STATE']].min()

AVG_SCORE    178.557611651728
STATE                 Alabama
dtype: object

In [ ]:
Select 
"State",
"Test Subject",
"Total Revenue",
MAX("Average Score") as "Max Avg Score",
Min("Average Score") as "Min Avg Score"
from education 
Group by "Total Revenue","State", "Test Subject"
Order by "Total Revenue" DESC;

In [ ]:
After looking at the data, there is no correlation between high funding within states and high test scores. 
California came out as the top highest state funding. Yet, it ranked 31 within the test scores. While Iowa scored 
highest in test scores and not even as high for funding. 